In [1]:
using Flux
using Flux: gradient
using Flux.Optimise: update!
using DelimitedFiles, Statistics
#using Parameters: @with_kw


In [2]:
rawdata = readdlm("data/test.csv")';


In [3]:
x = rawdata[1:19,:]
y = rawdata[20:20,:]

1×8 Matrix{Float64}:
 0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0

In [4]:
split_ratio = 0.7

0.7

In [5]:
split_index = floor(Int,size(x,2)*split_ratio)
x_train = x[:,1:split_index]
y_train = y[:,1:split_index]
x_test = x[:,split_index+1:size(x,2)]
y_test = y[:,split_index+1:size(x,2)]


1×3 Matrix{Float64}:
 0.0  1.0  1.0

In [6]:
y_train

1×5 Matrix{Float64}:
 0.0  0.0  0.0  1.0  1.0

In [7]:
tempy = []
for i in y_train
    if i == 0
        push!(tempy,0)
    else
        push!(tempy,1)
    end
    end
y_train = tempy

5-element Vector{Any}:
 0
 0
 0
 1
 1

In [8]:
tempy = []
for i in y_test
    if i == 0
        push!(tempy,0)
    else
        push!(tempy,1)
    end
    end
y_test = tempy

3-element Vector{Any}:
 0
 1
 1

In [10]:
y_train = Flux.onehotbatch(y_train, 0:1)


2×5 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 1  1  1  ⋅  ⋅
 ⋅  ⋅  ⋅  1  1

In [11]:
y_test = Flux.onehotbatch(y_test, 0:1)

2×3 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 1  ⋅  ⋅
 ⋅  1  1

1×5 Matrix{Float64}:
 0.0  0.0  0.0  1.0  1.0

In [14]:
train_data = (x_train, y_train)
test_data = (x_test, y_test)

([75.0 62.0 48.0; 11.4 10.9 10.3; … ; 0.0 0.1 0.1; 0.7 0.7 0.7], [0.0 1.0 1.0])

In [15]:
#Sihao Ren // sren03 // 947105531
using Flux
using Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using Flux: @epochs
using MLJBase
using Printf
using BSON: @save # for save weights
using Plots

In [12]:
model = Chain(Dense(19, 2*19, sigmoid), Dense(2*19, 2), softmax)

Chain(
  Dense(19, 38, σ),                     # 760 parameters
  Dense(38, 2),                         # 78 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 838 parameters, 3.523 KiB.

In [21]:
weights = rand(2*19,19)
bias = rand(2*19)
a = Dense(Flux.param(weights), Flux.param(bias), sigmoid)
model = Chain(a, Dense(2*19, 2), softmax)

UndefVarError: UndefVarError: param not defined

In [13]:
x_train

19×5 Matrix{Float64}:
 36.0  35.0  74.0  58.0  48.0
 27.4  26.9  15.3  11.6  11.5
  7.4   7.2   5.2   5.7   4.5
  2.6   2.0   2.0   2.3   1.6
  7.6   6.7   4.7   5.5   3.0
 34.7  29.6  42.2  42.6  52.4
  0.5   0.7   0.4   0.1   0.0
  2.1   2.8   1.7   0.5   0.1
 25.0  23.5  24.4  22.6   0.0
  1.6   2.6   0.9   0.9   1.3
  2.3   3.4   1.3   1.3   1.9
 69.9  76.5  67.0  68.9  67.4
  0.7   0.5   0.5   1.0   1.0
  3.4   2.0   1.7   0.9   1.5
  4.1   2.4   2.2   1.9   2.5
  1.9   3.7   1.0   0.8   0.3
  0.4   1.1   0.5   0.6   0.3
  0.4   0.5   0.3   0.1   0.4
  1.3   1.6   1.0   1.0   0.8

In [16]:
loss(x, y) = crossentropy(model(x), y) 
dataset = repeated((x_train,y_train),1);
optim = ADAM()
myaccuracy(x, y) = mean(onecold(model(x)) .== onecold(y))


myaccuracy (generic function with 1 method)

In [17]:
dataset

Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Matrix{Float64}, Flux.OneHotArray{UInt32, 2, 1, 2, Vector{UInt32}}}}}(Base.Iterators.Repeated{Tuple{Matrix{Float64}, Flux.OneHotArray{UInt32, 2, 1, 2, Vector{UInt32}}}}(([36.0 35.0 … 58.0 48.0; 27.4 26.9 … 11.6 11.5; … ; 0.4 0.5 … 0.1 0.4; 1.3 1.6 … 1.0 0.8], Bool[1 1 … 0 0; 0 0 … 1 1])), 1)

In [19]:
Flux.train!(loss, Flux.params(model), dataset, optim)